In [2]:
%load_ext autoreload
%autoreload 2

In [60]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from collections import OrderedDict

from adapters import *
from dataset import NumpyCsvDataSet
from runner import ExptRunner
from runner import SpecialExptRunner
# from networks.autoencoder import AutoEncoder
from networks.imageencoder import ImageEncoder
from networks.imagedecoder import ImageDecoder
from networks.imagedecoder import ImageDecoderFlatInput
from networks.composedautoencoder import ComposedAutoEncoder
from networks.DenseAutoEncoder import DenseAutoEncoder
from networks.ConvVae import ConvVae
from networks.vae import VAE
# from networks.etpr.autoencoder import autoencoder as etprautoenc
from networks.dense import Dense
from networks.dense import DenseForPolicy
from networks.lossfunctions import *
from networks.multinet import MultiNet
from networks.multinet import MultiEncoderNet
from helpers import ReportResults
import utils

In [4]:
opt = np.get_printoptions()
np.set_printoptions(threshold=np.inf)

In [5]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print (device)

cpu


In [6]:
start = time.time()
data_folder = 'data/demoplanner/version1'
v1_train_data = NumpyCsvDataSet(data_folder + '/train.csv', device=device)
v1_test_data = NumpyCsvDataSet(data_folder + '/test.csv', device=device)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 30.871170043945312 sec


In [101]:
start = time.time()
data_folder = 'data/demoplanner/version2'
v2_train_data = NumpyCsvDataSet(data_folder + '/train.csv', device=device)
v2_test_data = NumpyCsvDataSet(data_folder + '/test.csv', device=device)
print ("Time Taken: {} sec".format(time.time() - start))

Time Taken: 403.2247488498688 sec


In [7]:
train_data = v1_train_data
test_data = v1_test_data

In [61]:
print (train_data.data.shape)
print (test_data.data.shape)

print (train_data.data.device)
print (test_data.data.device)

print (train_data.data.dtype)
print (test_data.data.dtype)

torch.Size([13280, 2058])
torch.Size([1341, 2058])
cpu
cpu
torch.float32
torch.float32


In [115]:
pretrain_policy = Dense([4, 16, 16, 6], use_last_act=False, prefix='enc')
pretrain_policy.load_state_dict(torch.load('runs/04-08-10-24-50-Policy-V2-Dense-CE/autoenc.pth', 
                                  map_location=device))

pretrain_dynamics = Dense([4, 16, 2], prefix='enc')
pretrain_dynamics.load_state_dict(torch.load('runs/03-31-18-02-52-Dynamics-V1-Dense-SmoothL1/autoenc.pth', 
                                    map_location=device))

pretrain_imgEncDec = ComposedAutoEncoder(layers_channels=[16,16,16,16], useMaxPool=True, device=device)
pretrain_imgEncDec.load_state_dict(torch.load('runs/03-28-14-10-20-DemoPl-V2-ComposedAutoEncoderWithMaxPool-SmoothL1-/autoenc.pth',
                               map_location=device))

<All keys matched successfully>

In [ ]:
def load_mapped_state_dict(target, source, tPrefix, sPrefix):
    new_dict = target.state_dict()
    for p,v in source.state_dict().items():
        targetKey = p.replace(sPrefix, tPrefix)
        if targetKey in target.state_dict().keys():
            new_dict[targetKey] = v
    target.load_state_dict(new_dict)

In [104]:
net_builder = lambda: MultiNet(policy=Dense([4, 16, 16, 6], use_last_act=False),
                               dynamics=Dense([4, 16, 2]))

In [89]:
img_layers = [16,16,16,16]
z_dim = 64 # This should be set based on above img_layers
env_layers = [4, 16, z_dim]
in_channels = 1
img_encoder = lambda: ImageEncoder(in_channels, img_layers, 'imgenc', useMaxPool=True, addFlatten=True)
img_decoder = lambda: ImageDecoderFlatInput(z_dim, list(reversed(img_layers)), in_channels, 'imgdec', useSigmoid=True)
env_encoder = lambda: Dense(env_layers, use_last_act=False, prefix='envenc')
env_decoder = lambda: Dense(list(reversed(env_layers)), last_act='sigmoid', prefix='envdec')
combined = lambda l1, l2: nn.Sequential(OrderedDict([
                              ('EncNet', l1()),
                              ('DecNet', l2()),
                            ]))

In [118]:
n = net_builder()
print (next(n.policy.parameters()))
print (next(n.dynamics.parameters()))

Parameter containing:
tensor([[ 0.3559,  0.2051,  0.4631, -0.3775],
        [ 0.4513, -0.3036,  0.1314,  0.0291],
        [ 0.2754, -0.1573, -0.3654,  0.3527],
        [ 0.4962,  0.0783, -0.1249,  0.0386],
        [-0.4247,  0.0354, -0.0232, -0.3170],
        [-0.1845,  0.1161,  0.4885, -0.0458],
        [ 0.1966,  0.2223,  0.0383,  0.2507],
        [ 0.1853, -0.3856, -0.3157, -0.3622],
        [ 0.3128,  0.4363,  0.4375,  0.0511],
        [-0.4965,  0.0332,  0.0081, -0.3016],
        [ 0.1978,  0.2233,  0.1330,  0.0160],
        [-0.3559,  0.1614, -0.0499, -0.4496],
        [ 0.1967, -0.1545,  0.2343,  0.2660],
        [ 0.0617,  0.1519,  0.1517, -0.2094],
        [ 0.3149, -0.1406, -0.2972, -0.0117],
        [ 0.1319,  0.1511,  0.2159,  0.3919]], requires_grad=True)
Parameter containing:
tensor([[ 0.2971, -0.3855,  0.1434, -0.2455],
        [ 0.4880,  0.1804, -0.1671, -0.3590],
        [-0.3691, -0.0402, -0.2224,  0.4718],
        [ 0.3345,  0.0431, -0.2351,  0.2344],
        [-0.044

In [91]:
print (imgEncDec.state_dict().keys())
print (n.state_dict().keys())

odict_keys(['encoder.net.enc_conv0.weight', 'encoder.net.enc_conv0.bias', 'encoder.net.enc_conv1.weight', 'encoder.net.enc_conv1.bias', 'encoder.net.enc_conv2.weight', 'encoder.net.enc_conv2.bias', 'encoder.net.enc_conv3.weight', 'encoder.net.enc_conv3.bias', 'decoder.net.dec_convt0.weight', 'decoder.net.dec_convt0.bias', 'decoder.net.dec_convt1.weight', 'decoder.net.dec_convt1.bias', 'decoder.net.dec_convt2.weight', 'decoder.net.dec_convt2.bias', 'decoder.net.dec_convt3.weight', 'decoder.net.dec_convt3.bias'])
odict_keys(['EncNet.net.imgenc_conv0.weight', 'EncNet.net.imgenc_conv0.bias', 'EncNet.net.imgenc_conv1.weight', 'EncNet.net.imgenc_conv1.bias', 'EncNet.net.imgenc_conv2.weight', 'EncNet.net.imgenc_conv2.bias', 'EncNet.net.imgenc_conv3.weight', 'EncNet.net.imgenc_conv3.bias', 'DecNet.net.envdec_fc0.weight', 'DecNet.net.envdec_fc0.bias', 'DecNet.net.envdec_fc1.weight', 'DecNet.net.envdec_fc1.bias'])


In [122]:
def env_enc_pretrained_img_dec():
    net = combined(env_encoder, img_decoder)
    load_mapped_state_dict(net, pretrain_imgEncDec, 'DecNet.net.imgdec', 'decoder.net.dec')
    return net

def pretrained_img_enc_env_dec():
    net = combined(img_encoder, env_decoder)
    load_mapped_state_dict(net, pretrain_imgEncDec, 'EncNet.net.imgenc', 'encoder.net.enc')
    return net

def multinet_with_pretrained_policy_and_dynamics():
    net = MultiNet(policy=Dense([4, 16, 16, 6], use_last_act=False, prefix='enc'),
                   dynamics=Dense([4, 16, 2], prefix='enc'))
    net.policy.load_state_dict(pretrain_policy.state_dict())
    net.dynamics.load_state_dict(pretrain_dynamics.state_dict())
    return net

In [9]:
def run_expts(train_epochs):
    runners = []
    for i in range(len(expts)):
        inputs = expts[i]
        net = inputs[1]()
        net = net.to(device)
        runner = ExptRunner(inputs[0], net, train_data, test_data, 
                            inputs[2], inputs[3], inputs[4], data_to_img_func=inputs[5], device=device)
        optimizer = inputs[6](net) if inputs[6] else None
        train_and_test(runner, train_epochs, optimizer)
        runners.append(runner)

def train_and_test(runner, train_epochs, optimizer=None):
    print ("Experiment logs folder: {}".format(runner.log_folder))
    runner.train(train_epochs, optimizer, shouldShowReconstruction=False)
    runner.test()    

In [123]:
net = multinet_with_pretrained_policy_and_dynamics()
print (next(net.policy.parameters()))
print (next(net.dynamics.parameters()))

Parameter containing:
tensor([[ 2.1541e+00,  1.3347e+00, -1.5179e+00, -8.8442e-01],
        [-1.6891e+00, -1.3337e-01,  2.0345e+00,  8.4390e-01],
        [-1.2387e+00,  6.7191e-01,  1.8659e+00, -5.8997e-01],
        [-1.1722e+00, -1.4820e+00,  1.3749e+00,  1.2455e+00],
        [-1.4007e+00,  1.6215e+00,  1.4609e+00, -1.6081e+00],
        [ 1.1641e+00,  1.8706e+00, -9.6192e-01, -1.5849e+00],
        [-2.2746e-01, -1.3408e-01,  1.0522e-01,  3.3120e-01],
        [ 1.8129e+00,  1.8907e+00, -4.4704e-01, -9.0037e-01],
        [ 1.6567e+00, -1.1527e+00, -5.3506e-01,  1.9557e+00],
        [-1.1602e+00, -9.2956e-01,  1.3206e+00,  1.2268e+00],
        [-1.7325e+00,  1.7219e+00,  1.6757e+00, -1.6814e+00],
        [-3.3325e-01, -4.3241e-02,  2.2437e-02,  1.2709e-01],
        [ 1.7772e+00,  1.8430e+00, -1.8119e+00, -1.8316e+00],
        [ 1.3270e-01,  2.5540e-01, -2.1870e-01, -3.3522e-02],
        [-9.2949e-04, -1.1418e+00,  6.2241e-01,  2.0301e+00],
        [ 1.3402e+00, -1.8480e+00, -1.5349e+00, 

In [124]:
expts = [
    ['FineTunePolicyDynamics-V2-AdamOptimWith_LrMinus4-SmoothL1',
     multinet_with_pretrained_policy_and_dynamics,
     identity_adapter,
     smooth_l1_loss_adapter,
     dynamics_gradient_ground_truth_adapter,
     None,
     lambda net: optim.Adam(net.parameters(), lr=1e-4)],
    ['FineTunePolicyDynamics-V2-AdamOptimWith_LrMinus5-SmoothL1',
     multinet_with_pretrained_policy_and_dynamics,
     identity_adapter,
     smooth_l1_loss_adapter,
     dynamics_gradient_ground_truth_adapter,
     None,
     lambda net: optim.Adam(net.parameters(), lr=1e-5)],
    ['FineTunePolicyDynamics-V2-SGDOptimWith_LrMinus4_NoMomentum-SmoothL1', 
     multinet_with_pretrained_policy_and_dynamics,
     identity_adapter, 
     smooth_l1_loss_adapter, 
     dynamics_gradient_ground_truth_adapter, 
     None,
     lambda net: optim.SGD(net.parameters(), lr=1e-4)],
    ['FineTunePolicyDynamics-V2-SGDOptimWith_LrMinus5_NoMomentum-SmoothL1', 
     multinet_with_pretrained_policy_and_dynamics,
     identity_adapter, 
     smooth_l1_loss_adapter, 
     dynamics_gradient_ground_truth_adapter, 
     None,
     lambda net: optim.SGD(net.parameters(), lr=1e-5)],
    ['FineTunePolicyDynamics-V2-SGDOptimWith_LrMinus4_WithMomentum-SmoothL1', 
     multinet_with_pretrained_policy_and_dynamics,
     identity_adapter, 
     smooth_l1_loss_adapter, 
     dynamics_gradient_ground_truth_adapter, 
     None,
     lambda net: optim.SGD(net.parameters(), lr=1e-4, momentum=0.9)],
    ['FineTunePolicyDynamics-V2-SGDOptimWith_LrMinus5_WithMomentum-SmoothL1', 
     multinet_with_pretrained_policy_and_dynamics,
     identity_adapter, 
     smooth_l1_loss_adapter, 
     dynamics_gradient_ground_truth_adapter, 
     None,
     lambda net: optim.SGD(net.parameters(), lr=1e-5, momentum=0.9)],
]

In [125]:
train_epochs = 10
run_expts(train_epochs)

Experiment logs folder: runs/04-17-18-45-58-FineTunePolicyDynamics-V2-AdamOptimWith_LrMinus4-SmoothL1
Experiment logs folder: runs/04-17-18-47-06-FineTunePolicyDynamics-V2-AdamOptimWith_LrMinus5-SmoothL1
Experiment logs folder: runs/04-17-18-48-04-FineTunePolicyDynamics-V2-SGDOptimWith_LrMinus4_NoMomentum-SmoothL1
Experiment logs folder: runs/04-17-18-48-51-FineTunePolicyDynamics-V2-SGDOptimWith_LrMinus5_NoMomentum-SmoothL1
Experiment logs folder: runs/04-17-18-49-43-FineTunePolicyDynamics-V2-SGDOptimWith_LrMinus4_WithMomentum-SmoothL1
Experiment logs folder: runs/04-17-18-50-32-FineTunePolicyDynamics-V2-SGDOptimWith_LrMinus5_WithMomentum-SmoothL1


In [59]:
net = MultiEncoderNet().to(device)
runner = SpecialExptRunner('CombinedNet-V1-AlternateCombinationsPerEpoch-MSE', net,
                           train_data, test_data, device=device)
print ("Experiment logs folder: {}".format(runner.log_folder))
runner.train3(2)

Experiment logs folder: runs/04-20-19-09-42-CombinedNet-V1-AlternateCombinationsPerEpoch-MSE


In [21]:
epochs = 2
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)
# Training with one optimizer for all combinations
opt = optim.Adam(net.parameters())
crit = nn.MSELoss()
combinations = {
    0:['I', 'I'],
    1:['E', 'E'],
    2:['I', 'E'],
    3:['E', 'I']
}
for e in range(epochs):
    for i, data in enumerate(train_loader, 0):
        comb = combinations[i%4]
        net.setNetworksForForward(*comb)
        ip_adapter = It_scaled_adapter if comb[0] == 'I' else XtYt_scaled_adapter
        gt_adapter = It_scaled_adapter if comb[1] == 'I' else XtYt_scaled_adapter
        ip = ip_adapter(data)
        gt = gt_adapter(data)
        op = net(ip)
        loss = crit(op, gt)
        opt.zero_grad()
        loss.backward()
        opt.step()
        mBatchRem = i % 100
        if 0 <= mBatchRem <= 3:
            with torch.no_grad():
                ip = ip_adapter(test_data)
                gt = gt_adapter(test_data)
                op = net(ip)
                loss = crit(op, gt)
                print ('Loss on test set at epoch {}, mini batch index {} with combination {}: {}'.format(e, i, comb, loss.item()))


Loss on test set at epoch 0, mini batch index 0 with combination ['I', 'I']: 0.02987826243042946
Loss on test set at epoch 0, mini batch index 1 with combination ['E', 'E']: 0.019032321870326996
Loss on test set at epoch 0, mini batch index 2 with combination ['I', 'E']: 0.024042878299951553
Loss on test set at epoch 0, mini batch index 3 with combination ['E', 'I']: 0.029962103813886642
Loss on test set at epoch 0, mini batch index 100 with combination ['I', 'I']: 0.029654700309038162
Loss on test set at epoch 0, mini batch index 101 with combination ['E', 'E']: 0.014780594035983086
Loss on test set at epoch 0, mini batch index 102 with combination ['I', 'E']: 0.021364621818065643
Loss on test set at epoch 0, mini batch index 103 with combination ['E', 'I']: 0.0289880633354187
Loss on test set at epoch 1, mini batch index 0 with combination ['I', 'I']: 0.029054583981633186
Loss on test set at epoch 1, mini batch index 1 with combination ['E', 'E']: 0.013773190788924694
Loss on test se

In [126]:
def build_reporter(checkpoint_file, useL1Loss=False):
    rr = ReportResults(test_data.data, train_data.data, device,
                       checkpoint_file, useL1Loss)
    return rr

In [128]:
for checkpoint_file in utils.enumerate_files('runs/04-17/'):
    rr = build_reporter(checkpoint_file)
    print ('--------------------------------------------------------------')
    print (checkpoint_file)
    print ('--------------------------------------------------------------')
    rr.eval_test_samples()

--------------------------------------------------------------
runs/04-17/run1/04-17-16-18-22-EnvEncFixedImgDec-V2-Dense_ImageDecoderMaxPool-SmoothL1/train_checkpoint.tar
--------------------------------------------------------------
datapoint 0 test error 0.009120470844209194
datapoint 1 test error 0.005518916994333267
datapoint 2 test error 0.004766381811350584
datapoint 3 test error 0.005364737473428249
datapoint 4 test error 0.003259113524109125
--------------------------------------------------------------
runs/04-17/run1/04-17-18-33-10-FixedImgEncEnvDec-V2-ImageEncoderMaxPool_Dense-SmoothL1/train_checkpoint.tar
--------------------------------------------------------------
datapoint 0 test error 0.0002895758079830557
datapoint 1 test error 0.00011081941920565441
datapoint 2 test error 4.115376577829011e-05
datapoint 3 test error 3.9934377127792686e-05
datapoint 4 test error 9.88859756034799e-06
--------------------------------------------------------------
runs/04-17/run2/04-17-1

In [ ]:
print('Time,Goal,DataSet,Network,Loss(Training),TestLoss,TrainLoss')
for checkpoint_file in utils.enumerate_files('runs/04-17/'):
    rr = build_reporter(checkpoint_file)
    rr.eval_performance_metrics()

Time,Goal,DataSet,Network,Loss(Training),TestLoss,TrainLoss
04:17:16:18:22,EnvEncFixedImgDec,V2,Dense_ImageDecoderMaxPool,SmoothL1,0.004931663163006306,0.004976250696927309


In [31]:
rr.generate_csv_comparison_report(test_data_float, train_data_float,
                   rootdir='runs/04-07')

Time,Goal,DataSet,Network,Loss(Training),TestLoss,TrainLoss
04:07:17:58:07,Policy,V1,Dense,CE,1.2633792161941528,1.2391564846038818
04:07:19:03:09,Policy,V2,Dense,CE,1.1975611448287964,1.1838151216506958
04:07:19:16:20,Policy,V2,Dense,CE,1.1645100116729736,1.177262306213379
04:08:05:39:29,Policy,V2,DenseForPolicy,CE,1.4296996593475342,1.3790206909179688
04:08:05:48:18,Policy,V2,DenseForPolicy,CE,1.3616671562194824,1.3848485946655273
04:08:10:17:06,Policy,V2,Dense,CE,2.801307346089743e-06,2.7218900413572555e-06
04:08:10:24:50,Policy,V2,Dense,CE,1.96190711676536e-07,1.5136279785110673e-07


In [32]:
sample = torch.cat((test_data.data[1055:1060], test_data.data[1230:1235]), axis=0)
ground_truth = policy_groud_truth_class_adapter(sample).to(device)
for folder, net in rr.build_net(rootdir='runs/04-07/'):
    print ('Folder: ' + folder)
    op_batch, loss = rr.run_mini_batch(net, sample)
    x_loss = F.cross_entropy(op_batch[:, :3], ground_truth[:, 0], reduction='none')
    y_loss = F.cross_entropy(op_batch[:, 3:], ground_truth[:, 1], reduction='none')
    print (ground_truth)
    print (op_batch)
    print (loss)
    print (x_loss)
    print (y_loss)
    print ('-----------------------------------------')

Folder: 04-07-17-58-07-Policy-V1-Dense-CE
tensor([[0, 1],
        [0, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [2, 0],
        [2, 0],
        [2, 0],
        [2, 0],
        [2, 1]])
tensor([[9.9892e-01, 1.1768e-02, 4.4407e-06, 6.9097e-01, 4.3719e-08, 5.8148e-05],
        [9.8408e-01, 4.1556e-02, 1.0096e-03, 9.8178e-01, 5.3358e-06, 1.9832e-04],
        [1.0000e+00, 1.0968e-05, 2.3005e-34, 5.3218e-30, 2.1840e-32, 2.9180e-05],
        [1.0000e+00, 4.4320e-05, 4.1472e-32, 7.9823e-29, 2.2644e-30, 4.3043e-04],
        [9.9999e-01, 1.9323e-04, 6.4159e-30, 1.2560e-27, 2.0004e-28, 5.3637e-03],
        [2.7807e-11, 4.0247e-02, 9.9016e-01, 9.9007e-01, 3.9920e-02, 2.1634e-12],
        [1.2382e-08, 3.5409e-02, 9.9082e-01, 9.9139e-01, 3.6082e-02, 8.6834e-10],
        [4.4673e-06, 3.4880e-02, 9.9111e-01, 9.9073e-01, 3.2762e-02, 3.6731e-07],
        [1.6327e-03, 3.4472e-02, 9.9060e-01, 9.8911e-01, 2.6565e-02, 1.5856e-04],
        [0.0000e+00, 7.8143e-03, 9.9971e-01, 0.0000e+00, 9.

In [19]:
rr = ReportResults(map_netname_net, device, 
                   policy_input_adapter, policy_groud_truth_adapter,
                   loss_adapter_func = l1_loss_adapter)

In [20]:
rr.generate_csv_comparison_report(test_data.data, train_data.data,
                   rootdir='runs/04-08')

Time,Goal,DataSet,Network,Loss(Training),TestLoss,TrainLoss
04:08:07:35:53,Policy,V2,Dense,MSE,0.0009246765985153615,0.0009788279421627522
04:08:07:29:36,Policy,V2,Dense,MSE,0.015583701431751251,0.016864705830812454


In [21]:
sample = torch.cat((test_data.data[1055:1060], test_data.data[1230:1235]), axis=0)
ground_truth = policy_groud_truth_adapter(sample).to(device)
for folder, net in rr.build_net(rootdir='runs/04-08/'):
    print ('Folder: ' + folder)
    op_batch, loss = rr.run_mini_batch(net, sample)
    print (ground_truth)
    print (op_batch)
    print (loss)

Folder: 04-08-07-35-53-Policy-V2-Dense-MSE
tensor([[-1.,  0.],
        [-1., -1.],
        [-1.,  0.],
        [-1.,  0.],
        [-1.,  0.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1.,  0.]])
tensor([[-1.0000e+00, -5.9831e-04],
        [-9.9837e-01, -9.9866e-01],
        [-1.0000e+00, -2.0846e-03],
        [-1.0000e+00, -2.0014e-03],
        [-1.0000e+00, -1.9175e-03],
        [ 9.9764e-01, -9.9867e-01],
        [ 9.9765e-01, -9.9868e-01],
        [ 9.9766e-01, -9.9868e-01],
        [ 9.9767e-01, -9.9868e-01],
        [ 1.0000e+00,  7.1299e-04]], grad_fn=<TanhBackward>)
tensor(0.0012, grad_fn=<MeanBackward0>)
Folder: 04-08-07-29-36-Policy-V2-Dense-MSE
tensor([[-1.,  0.],
        [-1., -1.],
        [-1.,  0.],
        [-1.,  0.],
        [-1.,  0.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1., -1.],
        [ 1.,  0.]])
tensor([[-0.9994, -0.0311],
        [-0.9682, -0.9674],
        [-0.9995,  0.0176],
   

In [ ]:
np.set_printoptions(**opt)